In [0]:
%sql
CREATE TABLE IF NOT EXISTS workspace.gold_serving.dimension_systems (
  time TIMESTAMP,
  equipment_id STRING,
  im_id STRING,
  column_id STRING
) USING DELTA;

MERGE INTO workspace.gold_serving.dimension_systems AS target
USING (
  SELECT * FROM (
    SELECT
      time,
      equipment_id,
      im_id,
      column_id,
      ROW_NUMBER() OVER (
        PARTITION BY time, equipment_id, im_id
        ORDER BY time DESC
      ) AS rn
    FROM workspace.gold_staging.stg_systems
  ) sub
  WHERE rn = 1
) AS source
ON target.time = source.time
AND target.im_id = source.im_id
WHEN MATCHED THEN
  UPDATE SET
    target.equipment_id = source.equipment_id,
    target.column_id = source.column_id
WHEN NOT MATCHED THEN
  INSERT (
    time,
    equipment_id,
    im_id,
    column_id
  )
  VALUES (
    source.time,
    source.equipment_id,
    source.im_id,
    source.column_id
  );
